# Running BAGPIPES on a DJA NIRSpec/PRISM spectrum

Browse DJA v3 spectra here: https://s3.amazonaws.com/msaexp-nirspec/extractions/nirspec_graded_v3.html

In [1]:
# ------- LIBRARIES ------ #
import db_pull_funcs as db_pull
import pipes_fitting_funcs as p_fit
import pipes_plotting_funcs as p_plot

import bagpipes as pipes
import numpy as np

from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import Planck13 as cosmo

import os

In [2]:
if not os.path.exists("./files"):
    os.mkdir("./files")

### fitting params

In [3]:
def fitting_params(runid, z_spec, sfh="continuity", n_age_bins=10, scale_disp=1.3, dust_type="Calzetti", 
                   use_msa_resamp=False, fit_agn=False, fit_dla=False, fit_mlpoly=False):

    fit_instructions = {}
    
    ## ---------- ## double power-law sfh (parametric)
    dblplaw = {}                        
    dblplaw["tau"] = (0., 15.)            
    dblplaw["alpha"] = (0.01, 1000.)
    dblplaw["beta"] = (0.01, 1000.)
    dblplaw["alpha_prior"] = "log_10"
    dblplaw["beta_prior"] = "log_10"
    dblplaw["massformed"] = (1., 13.)
    dblplaw["metallicity"] = (0.003, 2.)
    dblplaw["metallicity_prior"] = "log_10"

    
    ## ---------- ## delayed-tau sfh (parametric)
    delayed = {}                         # Delayed Tau model t*e^-(t/tau)
    delayed["age"] = (0.1, 9.)           # Time since SF began: Gyr
    delayed["tau"] = (0.1, 9.)           # Timescale of decrease: Gyr
    delayed["massformed"] = (6., 13.)
    delayed["metallicity"] = (0.2, 1.)   # in Zsun

    
    # ## ---------- ## continuity sfh (non-parametric)
    def get_age_bins(z, n_age_bins=10):

        # sets max age to age of universe relative to age at z=30 (which is ~100 Myr after BB)
        max_age = cosmo.age(0).to('Myr').value - cosmo.age(30).to('Myr').value
        age_at_z = cosmo.age(z).to('Myr').value - cosmo.age(30).to('Myr').value

        age_bins = [0., 10., 50] # sets initial two age bin edges
        for i in np.logspace(np.log10(100), np.log10(max_age), n_age_bins):
            age_bins.append(i)
        age_bins=np.array(age_bins)

        # indeces of any age bin edges that are greater than max_age at z_obs
        last_index = np.where(age_bins > age_at_z)[0][0]

        # extends last age bin edge to age_at_z
        if (age_at_z-age_bins[last_index-1])<0.5*(age_bins[last_index]-age_bins[last_index-1]):
            age_bins[last_index-1] = age_at_z
            final_age_bins = age_bins[:last_index]
        elif (age_at_z-age_bins[last_index-1])>=0.5*(age_bins[last_index]-age_bins[last_index-1]):
            age_bins[last_index] = age_at_z
            final_age_bins = age_bins[:(last_index+1)]

        return final_age_bins
    
    continuity = {}
    continuity["massformed"] = (6., 14.)
    continuity["massformed_prior"] = "uniform"
    # continuity["metallicity"] = 0.5
    # continuity["metallicity"] = (0.5-0.1, 0.5+0.1)
    continuity["metallicity"] = (0.01, 2.5)
    continuity["metallicity_prior"] = "uniform"
    continuity["bin_edges"] = get_age_bins(z_spec, n_age_bins=n_age_bins).tolist()
    
    for i in range(1, len(continuity["bin_edges"])-1):
        continuity["dsfr" + str(i)] = (-10., 10.)
        continuity["dsfr" + str(i) + "_prior"] = "student_t"
        continuity["dsfr" + str(i) + "_prior_scale"] = 1.0  # Defaults to 0.3 as in Leja19, but can be set - 1 is bursty continuity prior from Tacchella+21
        continuity["dsfr" + str(i) + "_prior_df"] = 2       # Defaults to this value as in Leja19, but can be set
    continuity["age_min"] = 0

    # setting the preferred sfh
    if sfh=="continuity":
        fit_instructions["continuity"] = continuity
    elif sfh=="dblplaw":
        fit_instructions["dblplaw"] = dblplaw
    elif sfh=="delayed":
        fit_instructions["delayed"] = delayed
    
    ## ---------- ## nebular emisison, logU
    nebular = {}
    nebular["logU"] = (-4., -1.)
    nebular["logU_prior"] = "Gaussian"
    nebular["logU_prior_mu"] = -2.25
    nebular["logU_prior_sigma"] = 0.25
    fit_instructions["nebular"] = nebular
    
    ## ---------- ## dust law
    dust = {}
    if dust_type=="Calzetti":
        dust["type"] = "Calzetti"            # Shape of the attenuation curve
        dust["Av"] = (0., 6.)                # Vary Av between 0 and 4 magnitudes
    elif dust_type=="CF00":
        dust["type"] = "CF00"
        dust["eta"] = 2.
        dust["Av"] = (0., 6.0)
        dust["n"] = (0.3, 2.5)
        dust["n_prior"] = "Gaussian"
        dust["n_prior_mu"] = 0.7
        dust["n_prior_sigma"] = 0.3
    elif dust_type=="Salim":
        dust["type"] = "Salim"
        dust["Av"] = (0., 6.)                # Vary Av magnitude
        dust["delta"] = (-0.3, 0.3)          # Vary att. slope
        dust["delta_prior"] = "Gaussian"     # prior on att. slope
        dust["delta_prior_mu"] = 0           # avg. of prior on att. slope
        dust["delta_prior_sigma"] = 0.1      # standard dev. of prior on att. slope
        dust["B"] = (0., 3)                  # Vary 2175A bump strength
        dust["B_prior"] = "uniform"          # prior on 2175A bump strength
    elif dust_type=="Kriek":
        dust["type"] = "Salim"               # Specify parameters within the "Salim" model to match Kriek & Conroy 2013
        dust["Av"] = (0., 6.)                # Vary Av magnitude
        dust["eta"] = 2.                     # Multiplicative factor on Av for stars in birth clouds
        dust["delta"] = -0.2                 # Similar to Kriek & Conroy 2013
        dust["B"] = 1                        # Similar to Kriek & Conroy 2013
    fit_instructions["dust"] = dust

    ## ---------- ## max age of birth clouds: Gyr
    fit_instructions["t_bc"] = 0.01
    
    ## ---------- ## agn component
    agn = {}
    agn["alphalam"] = (-2., 2.)
    agn["betalam"] = (-2., 2.)
    agn["f5100A"] = (0, 1e-19)
    agn["sigma"] = (1e3, 5e3)
    agn["hanorm"] = (0,2.5e-17)
    if fit_agn:
        fit_instructions["agn"] = agn
    
    ## ---------- ## tight redshift prior around z_spec
    fit_instructions["redshift"] = (z_spec - 0.001*(1+z_spec), z_spec + 0.001*(1+z_spec))
    fit_instructions["redshift_prior"] = "Gaussian"
    fit_instructions["redshift_prior_mu"] = z_spec
    fit_instructions["redshift_prior_sigma"] = 0.001 * (1+z_spec)

    ## ---------- ## jwst prism resolution curve
    hdul = fits.open("jwst_nirspec_prism_disp.fits")
    resData = np.c_[10000*hdul[1].data["WAVELENGTH"], hdul[1].data["R"]*scale_disp]
    fit_instructions["R_curve"] = resData

    ## ---------- ## boolean for using msa resampling
    fit_instructions["use_msa_resamp"] = use_msa_resamp

    ## ---------- ## fixed velocity dispersion
    fit_instructions["veldisp"] = 100.   # km/s

    ## ---------- ## dla component (doesn't work?)
    dla = {}
    dla["zabs"] = z_spec
    dla["t"] = 22.
    if fit_dla:
        fit_instructions["dla"] = dla

    ## ---------- ## calibration curve (2nd order polynomial)
    cfit, cfit_err, _ = p_fit.guess_calib(runid, z=z_spec) # guessing initial calibration coefficients

    calib = {}
    calib["type"] = "polynomial_bayesian"
    
    calib["0"] = (cfit[0]-5.*cfit_err[0], cfit[0]+5.*cfit_err[0])
    calib["0_prior"] = "Gaussian"
    calib["0_prior_mu"] = cfit[0]
    calib["0_prior_sigma"] = cfit_err[0]
    
    calib["1"] = (cfit[1]-5.*cfit_err[1], cfit[1]+5.*cfit_err[1])
    calib["1_prior"] = "Gaussian"
    calib["1_prior_mu"] = cfit[1]
    calib["1_prior_sigma"] = cfit_err[1]
    
    calib["2"] = (cfit[2]-5.*cfit_err[2], cfit[2]+5.*cfit_err[2])
    calib["2_prior"] = "Gaussian"
    calib["2_prior_mu"] = cfit[2]
    calib["2_prior_sigma"] = cfit_err[2]
    fit_instructions["calib"] = calib
    
    # ## ---------- ##
    if fit_mlpoly:
        mlpoly = {}
        mlpoly["type"] = "polynomial_max_like"
        mlpoly["order"] = 4
        fit_instructions["calib"] = mlpoly

    ## ---------- ## white noise scaling
    noise = {}
    noise["type"] = "white_scaled"
    noise["scaling"] = (1., 10.)
    noise["scaling_prior"] = "log_10"
    fit_instructions["noise"] = noise

    return(fit_instructions)

In [4]:
fnames = ['rubies-egs61-v3_prism-clear_4233_75646',
          'gds-deep-v3_prism-clear_1210_9297',
          'rubies-uds31-v3_prism-clear_4233_166283',
          'gds-barrufet-s67-v3_prism-clear_2198_6620',
          'ceers-ddt-v3_prism-clear_2750_28',
          'rubies-egs51-v3_prism-clear_4233_19489',
          'gds-barrufet-s67-v3_prism-clear_2198_5446',
          'rubies-uds3-v3_prism-clear_4233_62812',
          'ceers-ddt-v3_prism-clear_2750_9600',
          'uncover-61-v3_prism-clear_2561_13416',
          'ceers-v3_prism-clear_1345_397', 
          'rubies-uds1-v3_prism-clear_4233_32052',
          'rubies-uds41-v3_prism-clear_4233_15795']
ids = [0,1,2,3,4,5,6,7,8,9,10,11,12]

# catalogue of spectra from DJA
speclist_cat = Table([ids, fnames], names=list(['id', 'fname']))
speclist_cat.write('spec_cat_temp.fits', format='fits', overwrite=True)

In [ ]:
def run_pipes_on_dja_spec(spec_cat):
    """
    Runs bagpipes on spectrum from DJA AWS database, saves posteriors as .h5 files and plots as .pdf files
    
    Parameters
    ----------
    spec_cat : catalogue of spectra and corresponding ids, format=astropy Table

    Returns
    -------
    None
    """

    # name of bagpipes run
    runName = 'pipes_test'
    
    # make pipes folders if they don't already exist
    pipes.utils.make_dirs(run=runName)

    ##################################
    # ---- PULLING DATA FROM DB ---- #
    ##################################

    # id and dja name of spectrum
    runid, fname = spec_cat

    print(fname)

    # spectrum and photometry filenames
    fname_spec = fname+'.spec.fits'
    fname_phot = fname+'.phot.cat'

    # path to store spectrum and photometry files
    filePath = 'files/'

    # pull spectrum and photometry from the aws database
    db_pull.pull_spec_from_db(fname_spec, filePath)

    try:
        db_pull.pull_phot_from_db(fname_spec, fname_phot, filePath)
    except IndexError:
        print("No photometry found")
        return None
    

    # spectroscopic redshift
    z_spec = db_pull.pull_zspec_from_db(fname_spec)

    # save data to plot
    p_plot.plot_spec_phot_data(fname_spec, fname_phot, z_spec=z_spec, f_lam=True, show=False, save=True, run=runName)

    ##################################
    # -------- BAGPIPES RUN -------- #
    ##################################

    # jwst filter list
    filt_list = p_fit.updated_filt_list(runid)#np.loadtxt("../filters/filt_list.txt", dtype="str")

    # making galaxy object
    galaxy = pipes.galaxy(runid, p_fit.load_both, filt_list=filt_list,
                        spec_units='ergscma',
                        phot_units='ergscma',
                        out_units="ergscma")

    # generating fit instructions
    fit_instructions = fitting_params(runid, z_spec, sfh="continuity", dust_type="Kriek", fit_mlpoly=False)

    # making fit object
    fit = pipes.fit(galaxy, fit_instructions, run=runName)

    # fitting  spectrum and photometry with bagpipes
    fit.fit(verbose=False, sampler='nautilus', pool=10)

    ##################################
    # ---------- PLOTTING ---------- #
    ##################################

    # fitted model
    fig = p_plot.plot_fitted_spectrum(fit, fname_spec, z_spec=z_spec, f_lam=True, save=True)
    # star-formation history
    fig = p_plot.plot_fitted_sfh(fit, fname_spec, z_spec=z_spec, save=True)
    # posterior corner plot
    # fig = p_plot.plot_corner(fit, fname_spec, z_spec=z_spec, fit_instructions, filt_list, save=True)
    fig = p_plot.plot_corner(fit, fname_spec, z_spec, fit_instructions, filt_list, save=True)
    # calibration curve
    fig = p_plot.plot_calib(fit, fname_spec, z_spec=z_spec, save=True)

    # saving posterior quantities to table
    tab = p_plot.save_posterior_sample_dists(fit, fname_spec, save=True)
    # saving calib curve to table
    tab = p_plot.save_calib(fit, fname_spec, save=True)

    return fit

In [6]:
# runids = [0,1,2,3,4,5,6,7,8,10]
# for i in runids:
#     fit = run_pipes_on_dja_spec(speclist_cat[i])

fit = run_pipes_on_dja_spec(speclist_cat[2])

rubies-uds31-v3_prism-clear_4233_166283

Results loaded from pipes/posterior/pipes_test/2.h5

Fitting not performed as results have already been loaded from pipes/posterior/pipes_test/2.h5. To start over delete this file or change run.

